# Voxel pipeline

### Import requirements

In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np

from ImageProcessor import ImageProcessor
from StereoMatcher import StereoMatcher
from helperScripts.TimeKeeper import TimeKeeper
from VoxelGrid import VoxelGrid

%matplotlib ipympl

### Load calibrations and other data

In [2]:
imageProcessor = ImageProcessor()
imageProcessor.verbose = True
imageProcessor.loadMonoCalibration()
imageProcessor.loadStereoCalibration()
imageProcessor.loadCameraProperties()
imageProcessor.loadStereoRectify()

Reading from data/monoCalibration.json
Loaded mono calibration
Reading from data/stereoCalibration.json
Loaded stereo calibration
Reading from data/cameraProperties.json
Loaded camera properties
Reading from data/stereoRectify.json
Loaded stereo rectification data


### Create stereo matcher

In [3]:
stereoMatcher = StereoMatcher(imageProcessor=imageProcessor, \
                matcher="SGBM", vertical=True, createRightMatcher=False)

imageProcessor.initUndistortRectifyMap()
#stereoMatcher.createDisparityWLSFilter()

Reading from data/parametersSGBM.json


### Timekeeper for performance metrics

In [4]:
timeKeeper = TimeKeeper()

### Loading images

In [5]:
path = "testImages/voxelTestImages"
imageGlobL = sorted(glob.glob("".join([path, "/top_*", ".png"])))
imageGlobR = sorted(glob.glob("".join([path, "/bottom_*", ".png"])))
print ("Selections: 0-{}".format(len(imageGlobL)-1))

Selections: 0-0


### Select image pair and display

In [6]:
imageStart = 0
imageSelection = [imageStart, imageStart+1]

def loadImages(imageNumber):
    imageL = cv2.imread(imageGlobL[imageNumber])
    imageR = cv2.imread(imageGlobR[imageNumber])

    return imageL, imageR

imageL, imageR = loadImages(imageSelection[0])

plt.figure()
plt.imshow(cv2.cvtColor(np.hstack([imageL, imageR]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Convert to grayscale and undistort

In [7]:
imageProcessor.convertToGrayscale(imageL, imageR)
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

### View undistorted image

In [8]:
fig = plt.figure()
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(np.hstack([imageProcessor.undistortImageL, \
            imageProcessor.undistortImageR]), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig = plt.figure()
fig.suptitle("horizontal epipolar")
plt.imshow(cv2.cvtColor(imageProcessor.drawHorEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
fig = plt.figure(figsize=(6,15))
fig.suptitle("left/right undistorted")
plt.imshow(cv2.cvtColor(imageProcessor.drawVertEpipolarLines(\
        imageProcessor.undistortImageL, imageProcessor.undistortImageR), cv2.COLOR_BGR2RGB))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute disparity map

In [11]:
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()
#stereoMatcher.applyWLSFilterDisparity()

print ("minDisparity:", stereoMatcher.disparityMapL.min())
print ("maxDisparity:", stereoMatcher.disparityMapL.max())

minDisparity: 9.0
maxDisparity: 41.0


### View disparity map

In [12]:
plt.figure()
plt.imshow(cv2.rotate(stereoMatcher.disparityMapL, \
    cv2.ROTATE_90_CLOCKWISE))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Compute depth map

In [13]:
focalLength = imageProcessor.projectionMatrixL[0][0] # changes with rectify?
baseline = 32 # mm, measured irl

stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==0] = 0.9
stereoMatcher.disparityMapL[stereoMatcher.disparityMapL==-1] = 0.9

depthMap = np.empty_like(stereoMatcher.disparityMapL)
depthMap = (focalLength*baseline)/stereoMatcher.disparityMapL[:]

print (stereoMatcher.disparityMapL.min())
print (stereoMatcher.disparityMapL.max())
print (depthMap.shape)
print (depthMap.min())
print (depthMap.max())

9.0
41.0
(640, 360)
481.33795
2192.7617


### View depth map

In [14]:
plt.figure()
plt.imshow(cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# Use to save images
# _=cv2.imwrite("depth.png", cv2.rotate(depthMap, cv2.ROTATE_90_CLOCKWISE))

### Compute point cloud

In [16]:
pointSubsample = 24
voxelSize = 100
voxelStopFraction = 10
occupancyThreshold = 10

In [17]:
timeKeeper.startPerfCounter()

def generatePointCloud(disparityMapL, dispToDepthMatrix):
    points = cv2.reprojectImageTo3D(\
            disparityMapL, \
            dispToDepthMatrix)

    # Reshaping to a list of 3D coordinates
    pointCloud = points.reshape(\
                (points.shape[0]*points.shape[1],3))[0::pointSubsample]\
                                                .astype(np.int16)

    return pointCloud

pointCloud = generatePointCloud(stereoMatcher.disparityMapL,\
                                imageProcessor.dispToDepthMatrix)

print("".join(["Points in unfiltered pointcloud: {}; ",\
                    "completed in {:.5f} sec"]).format(\
                    pointCloud.shape[0], \
                    timeKeeper.returnPerfCounter()))

Points in unfiltered pointcloud: 9600; completed in 0.00264 sec


Filtering extreme points

In [18]:
timeKeeper.startPerfCounter()

def filterPointCloud(pointCloud):
    # Filtering x values
    pointCloud = pointCloud[np.logical_and(\
            pointCloud[:, 0]>pointCloud[:, 0].min(), \
            pointCloud[:, 0]<pointCloud[:, 0].max())]
    # Filtering y values
    pointCloud = pointCloud[np.logical_and(\
            pointCloud[:, 1]>pointCloud[:, 1].min(), \
            pointCloud[:, 1]<pointCloud[:, 1].max())]
    # Filtering z values
    pointCloud = pointCloud[np.logical_and(\
            pointCloud[:, 2]>pointCloud[:, 2].min(), \
            pointCloud[:, 2]<pointCloud[:, 2].max())]

    return pointCloud

pointCloud = filterPointCloud(pointCloud)

print("".join(["Points in filtered pointcloud: {}; ",\
                "completed in {:.5f} sec"]).format(\
                pointCloud.shape[0], \
                timeKeeper.returnPerfCounter()))

Points in filtered pointcloud: 8006; completed in 0.00086 sec


Rotate point cloud to have y forward

In [19]:
redefineRotationMatrix = np.array([ [ 0,  0, -1],
                                    [ 0,  1,  0],
                                    [ 1,  0,  0] ])

def rotatePointCloud(pointCloud, rotationMatrix):
    pointCloud = np.dot(pointCloud[:], rotationMatrix)

    return pointCloud

pointCloud = rotatePointCloud(pointCloud, redefineRotationMatrix)

### View point cloud

In [20]:
def plotGrid(grid, s):
    fig = plt.figure(figsize=(7,7))
    ax = fig.add_subplot(111, projection = "3d")

    ax.scatter(grid[:,0], grid[:,1], grid[:,2], s=s)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    ax.set_zlabel("$z$")

    # Camera axis begins at x=0 and looks to positive x
    ax.set_xlim(0,2000)
    ax.set_ylim(-1000,1000)
    ax.set_zlim(-1000,1000)

    plt.show()

plotGrid(pointCloud, 1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Voxelize point cloud

In [21]:
voxelGrid = None

timeKeeper.startPerfCounter()

def voxelizePointCloud(pointCloud, voxelSize, \
                        occupancyThreshold, voxelStopFraction):
    iterations = 0
    voxelGrid = []
    initialSize = pointCloud.shape[0]
    remainingPoints = initialSize
    samplingLimit = np.zeros_like(pointCloud[0])
        
    while remainingPoints>(initialSize/voxelStopFraction):

        sampledPoint = pointCloud[np.random.randint(0,remainingPoints)]

        for n in range(3):
            samplingLimit[n]=\
                (sampledPoint[n]//voxelSize)*voxelSize

        mask = np.ones(remainingPoints, dtype=bool)

        for n in range(len(sampledPoint)):
            mask = np.logical_and(mask, np.logical_and(\
                pointCloud[:,n]>=samplingLimit[n], \
                pointCloud[:,n]<samplingLimit[n]+voxelSize))

        pointsInVoxel = pointCloud[mask]

        if len(pointsInVoxel)>occupancyThreshold:
            voxelMidpoint = samplingLimit+voxelSize/2
            voxelGrid.append(voxelMidpoint)

        pointCloud = pointCloud[np.invert(mask)]

        iterations+=1

        remainingPoints = pointCloud.shape[0]

    voxelGrid = np.array(voxelGrid, dtype=np.int16)

    ### These steps will potentially need to be changed

    if voxelGrid is None:
        voxelGrid = voxelGrid

    else:
        voxelGrid = np.unique(np.vstack((voxelGrid, voxelGrid)), axis=0)

    ###

    return voxelGrid, iterations

voxelGrid, iterations = voxelizePointCloud(pointCloud, voxelSize, \
                        occupancyThreshold, voxelStopFraction)

print("".join(["Voxels in grid: {}; ",\
                "completed in {:.5f} sec; {} iterations"]).format(\
                voxelGrid.shape[0], \
                timeKeeper.returnPerfCounter(), \
                iterations))

Voxels in grid: 65; completed in 0.00868 sec; 70 iterations


### View voxelized point cloud

In [22]:
plotGrid(voxelGrid, 100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Refine voxel grid with newer data

Interchanging data from calibration since calibration was 
done vertically

The x y notations are in image coordinates

In [23]:
# Horizontal field of view (degrees)
fovH = ((imageProcessor.fovYL+imageProcessor.fovYR)/4)*np.pi/180
fovH -= fovH/8
print("Horizontal:", fovH*180/np.pi)

# Vertical field of view (degrees)
fovV = ((imageProcessor.fovXL+imageProcessor.fovXR)/4)*np.pi/180
fovV -= fovV/8
print("Vertical:", fovV*180/np.pi)

Horizontal: 24.915932084055157
Vertical: 14.867033874651561


### Checking for voxels in range

The refinement is done by simply replacing the older voxels in view of the camera in the base grid with the new voxels.

Here the voxels that may potentially be affected are found.

In [24]:
# Do this after compensations for atleast camera translation have been made
# Performed on the base voxel grid
# Distances to every voxel in base grid from camera

# Offset camera position with distance from robot center and distance of robot 
# from origin
# The top or left camera is taken to represent the whole camera grid
cameraPosition = np.zeros([3])
distanceToVoxels = np.linalg.norm(voxelGrid-cameraPosition, axis=1)
print(distanceToVoxels.shape)

# Distance within which points may be modified
distanceToCheck = 1500

# These are the points to check
# Performed on the base voxel grid
voxelsWithinRange = voxelGrid[distanceToVoxels<=distanceToCheck]
print(voxelGrid.shape)
print(voxelsWithinRange.shape)

# Checking bounds on the coordinate axes
# Performed on the base voxel grid
voxelCheckBound = [[voxelsWithinRange[:,0].min(), voxelsWithinRange[:,1].min(), \
                        voxelsWithinRange[:,2].min()], \
                    [voxelsWithinRange[:,0].max(), voxelsWithinRange[:,1].max(), \
                        voxelsWithinRange[:,2].max()]]
print(voxelCheckBound) # min, max

(65,)
(65, 3)
(65, 3)
[[550, -650, -250], [1250, 250, 250]]


### Generating new voxel grid from newer data

In [25]:
# Load next image pair
imageL, imageR = loadImages(imageSelection[1])

# Undistort images
imageProcessor.convertToGrayscale(imageL, imageR)
imageProcessor.undistortRectifyRemap(imageProcessor.grayImageL, \
                                        imageProcessor.grayImageR)

# Compute disparity map
stereoMatcher.computeDisparity(\
                grayImageL=imageProcessor.undistortImageL, \
                grayImageR=imageProcessor.undistortImageR)

stereoMatcher.clampDisparity()
stereoMatcher.applyClosingFilter()

# Compute new point cloud
pointCloud = generatePointCloud(stereoMatcher.disparityMapL,\
                                imageProcessor.dispToDepthMatrix)

# Filter point cloud
pointCloud = filterPointCloud(pointCloud)

# Rotate point cloud
pointCloud = rotatePointCloud(pointCloud, redefineRotationMatrix)

# Generate new voxel grid
newVoxelGrid, iterations = voxelizePointCloud(pointCloud, voxelSize, \
                        occupancyThreshold, voxelStopFraction)

IndexError: list index out of range

### View new voxel grid

In [45]:
plotGrid(newVoxelGrid, 100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Angle ranges of rotated new voxel grid (Exploration only)

Could potentially be found from the rotation matrix and be offset 
by the camera fovs.

Computing yaws on the new voxel grid; xy plane. A line along x axis has 0 degrees of yaw.

Normally this would be the results of the latest iteration after 
they are rotated and translated.

Taking x forward

In [25]:
yawToNewVoxels = np.arctan2(newVoxelGrid[:,1], newVoxelGrid[:,0]) # y, x
print(yawToNewVoxels.shape)

yawCheckBound = np.array([yawToNewVoxels.max(), yawToNewVoxels.min()]) # left, right
print(yawCheckBound*180/np.pi)
print((yawCheckBound[0]-yawCheckBound[1])*180/np.pi)

#print(yawToNewVoxels*180/np.pi)

(61,)
[ 24.443953 -27.47443 ]
51.918385239909625


Computing pitches on the new voxel grid; xz plane.

Taking x forward

In [26]:
pitchToNewVoxels = np.arctan2(newVoxelGrid[:,2], newVoxelGrid[:,0])
print(pitchToNewVoxels.shape)

                                                                # up, down
pitchCheckBound = np.array([pitchToNewVoxels.max(), pitchToNewVoxels.min()]) 
print(pitchCheckBound*180/np.pi)
print((pitchCheckBound[0]-pitchCheckBound[1])*180/np.pi)

#print(pitchToNewVoxels*180/np.pi)

(61,)
[ 15.255118 -16.38954 ]
31.64465916054635


Computing rolls on the new voxel grid; yz plane.

Taking x forward

In [27]:
rollToNewVoxels = np.arctan2(newVoxelGrid[:,2], newVoxelGrid[:,1])
print(rollToNewVoxels.shape)

rollCheckBound = np.array([rollToNewVoxels.max(), rollToNewVoxels.min()])
print(rollCheckBound*180/np.pi)
print((rollCheckBound[0]-rollCheckBound[1])*180/np.pi)

#print(rollToNewVoxels*180/np.pi)

(61,)
[ 171.86989 -174.80557]
346.67548573975506


From the results above, checking for pitch and roll will return 
inconclusive results unless the rotation is in an exact side profile of the
grid.

This can be mitigated by doing the check before rotating or translating, but that would require the base grid to be rotated as a whole, refined, and then re-rotated back.

To save on computation, the yaw range for the new voxel grid may simply be taken from camera properties already found during calibration. 

Then, to check where the camera is facing, a vector of unit distance from the camera facing along its axis could be rotated and translated along with the camera. 

Its yaw with respect to the camera center can then be calculated, and then offset by half its horizontal fov on both sides, to find the yaw limits within which the base voxels may be modified.

### Finding yaw of camera and voxels within range
Finding yaw of camera

In [28]:
# Camera vector before rotation, z forward
cameraDirectionVector = np.array([0,0,100])
print(cameraDirectionVector)

# Camera vector after rotation (do not translate)
# Also perform rotations to align with the robot frame, not done here
cameraDirectionVector = np.dot(\
            cameraDirectionVector, redefineRotationMatrix)
print(cameraDirectionVector)

# Finding yaw of the camera vector (again, assuming no translation)
cameraYaw = np.arctan2(cameraDirectionVector[1], cameraDirectionVector[0])
print(cameraYaw*180/np.pi)

cameraYawRange = np.array([cameraYaw+fovH, cameraYaw-fovH])
print(cameraYawRange*180/np.pi)

# Wrapping around values at -180, 180 degrees
for n in range(len(cameraYawRange)):
    if cameraYawRange[n]>np.pi:
        cameraYawRange[n] -= 2*np.pi
    if cameraYawRange[n]<=-np.pi:
        cameraYawRange[n] += 2*np.pi

cameraYawRange = np.sort(cameraYawRange)[::-1]

print(cameraYawRange*180/np.pi)

[  0   0 100]
[100   0   0]
0.0
[ 0.43486505 -0.43486505]
[ 24.91593208 -24.91593208]
[ 0.43486505 -0.43486505]
[ 24.91593208 -24.91593208]


Finding yaw of voxels within range

In [29]:
# Shifting the voxels around the camera to origin so that yaw to each voxel 
# can be found
translatedVoxelsWithinRange = voxelsWithinRange-cameraPosition
print(translatedVoxelsWithinRange.shape)

yawToTranslatedVoxels = np.arctan2(translatedVoxelsWithinRange[:,1], \
                                translatedVoxelsWithinRange[:,0])
print(yawToTranslatedVoxels.shape)
#print(yawToTranslatedVoxels*180/np.pi)

(60, 3)
(60,)


Finding voxels that need to be removed

In [30]:
if cameraYawRange[0]>np.pi/2 and cameraYawRange[1]<-np.pi/2:
    voxelsToRemove = voxelsWithinRange[np.logical_and(\
        yawToTranslatedVoxels[:]>cameraYawRange[0], \
        yawToTranslatedVoxels[:]<cameraYawRange[1]
        )]
else:
    voxelsToRemove = voxelsWithinRange[np.logical_and(\
        yawToTranslatedVoxels[:]<cameraYawRange[0], \
        yawToTranslatedVoxels[:]>cameraYawRange[1]
        )]

print(voxelsToRemove.shape)

(58, 3)


The voxels to be removed are then removed from the base grid

In [31]:
print(voxelGrid.shape)
print(voxelsToRemove.shape)

# Found this online
def in1d_dot_approach(A,B):
    """Returns the first array with elements from the second array removed"""
    cumdims = (np.maximum(A.max(),B.max())+1)**np.arange(B.shape[1])
    return A[~np.in1d(A.dot(cumdims),B.dot(cumdims))]

voxelRemovedGrid = in1d_dot_approach(voxelGrid, voxelsToRemove)
print(voxelRemovedGrid.shape)
print(voxelRemovedGrid)

(61, 3)
(58, 3)
(3, 3)
[[1150 -550  -50]
 [1250 -650  250]
 [1350 -650  250]]


Combining unique voxels from voxel-removed base grid and the new voxel grid to get the updated grid

In [32]:
updatedVoxelGrid = np.unique(\
                    np.vstack((voxelRemovedGrid, newVoxelGrid)), axis=0)
print(updatedVoxelGrid.shape)
voxelGrid = updatedVoxelGrid
print(voxelGrid.shape)

(61, 3)
(61, 3)
